In [18]:
import pandas as pd
import datetime

In [3]:
videos_df = pd.read_csv("outputs/videoInfo.csv")
videos_df["shorts"] = videos_df["title"].map(lambda x: 1 if "#shorts" in x else 0)

In [44]:
numeric_selected_vars = [
    "channelTitle",
    "duration",
    "viewCount",
    "likeCount",
    "commentCount",
    "publishedAtDay",
    "publishedAtHour"
    ]

numeric_df = videos_df[numeric_selected_vars].copy()
numeric_df["percInteraction"] = numeric_df["commentCount"]/numeric_df["viewCount"]
numeric_df["percLike"] = numeric_df["likeCount"]/numeric_df["viewCount"]
numeric_df["ratioLikeComment"] = numeric_df["likeCount"]/numeric_df["commentCount"]

numeric_df.head()

,channelTitle,duration,viewCount,likeCount,commentCount,publishedAtDay,publishedAtHour,percInteraction,percLike,ratioLikeComment
0,Graham Stephan,896.0,25981.0,2109.0,389.0,2022-07-11,22,0.014972,0.081175,5.421594
1,Graham Stephan,727.0,440063.0,20145.0,1841.0,2022-07-08,22,0.004183,0.045778,10.942423
2,Graham Stephan,794.0,336090.0,14612.0,1498.0,2022-07-06,21,0.004457,0.043476,9.754339
3,Graham Stephan,775.0,677600.0,29372.0,1602.0,2022-07-01,20,0.002364,0.043347,18.334582
4,Graham Stephan,782.0,1133545.0,47011.0,2762.0,2022-06-29,21,0.002437,0.041473,17.020637


In [49]:
minCommonDate = numeric_df.groupby("channelTitle").agg({"publishedAtDay": "min"}).publishedAtDay.max()
maxCommonDate = numeric_df.groupby("channelTitle").agg({"publishedAtDay": "max"}).publishedAtDay.max()

raw_df = numeric_df[numeric_df.publishedAtDay >= minCommonDate].drop(columns = ["channelTitle"])
raw_df["publishedAtDay"] = raw_df["publishedAtDay"].map(lambda x: pd.to_datetime(x))
raw_df["weekNumber"] = raw_df["publishedAtDay"].map(lambda x: x.strftime("%V"))

matchDayWeek = raw_df[["publishedAtDay", "weekNumber"]].drop_duplicates().sort_values("publishedAtDay", ascending = False)
matchDayWeek["weekId"] = range(matchDayWeek.shape[0])
matchDayWeek = matchDayWeek[["weekId", "publishedAtDay"]].reset_index(drop = True)

raw_df = raw_df.drop(columns = ["weekNumber"]).merge(matchDayWeek, on = ["publishedAtDay"])
raw_df

,duration,viewCount,likeCount,commentCount,publishedAtDay,publishedAtHour,percInteraction,percLike,ratioLikeComment,weekId
0,896.0,25981.0,2109.0,389.0,2022-07-11,22,0.014972,0.081175,5.421594,0
1,836.0,9855.0,821.0,171.0,2022-07-11,22,0.017352,0.083308,4.801170,0
2,727.0,440063.0,20145.0,1841.0,2022-07-08,22,0.004183,0.045778,10.942423,1
3,867.0,350714.0,16596.0,3176.0,2022-07-08,22,0.009056,0.047321,5.225441,1
4,794.0,336090.0,14612.0,1498.0,2022-07-06,21,0.004457,0.043476,9.754339,3
...,...,...,...,...,...,...,...,...,...,...
946,916.0,201434.0,4307.0,278.0,2019-03-29,17,0.001380,0.021382,15.492806,566
947,297.0,13647.0,425.0,66.0,2019-02-05,7,0.004836,0.031142,6.439394,590
948,390.0,32877.0,1148.0,99.0,2019-01-29,6,0.003011,0.034918,11.595960,594
949,279.0,28061.0,1050.0,92.0,2019-01-22,2,0.003279,0.037418,11.413043,598
